In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os

In [ ]:
class MaskDetectionTrainer:
    def __init__(self, data_dir, img_size=(224, 224)):
        """
        Initialize the trainer
        data_dir should have structure:
        data_dir/
            train/
                with_mask/
                without_mask/
            validation/
                with_mask/
                without_mask/
        """
        self.data_dir = data_dir
        self.img_size = img_size
        self.model = None
        
    def create_model(self):
        """Create model using MobileNetV2 as base"""
        base_model = MobileNetV2(
            input_shape=(*self.img_size, 3),
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False
        
        model = keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(2, activation='softmax')  # 2 classes: with_mask, without_mask
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        return model
    
    def prepare_data(self, batch_size=32):
        """Prepare training and validation data"""
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            zoom_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        
        val_datagen = ImageDataGenerator(rescale=1./255)
        
        train_generator = train_datagen.flow_from_directory(
            os.path.join(self.data_dir, 'train'),
            target_size=self.img_size,
            batch_size=batch_size,
            class_mode='categorical'
        )
        
        val_generator = val_datagen.flow_from_directory(
            os.path.join(self.data_dir, 'validation'),
            target_size=self.img_size,
            batch_size=batch_size,
            class_mode='categorical'
        )
        
        return train_generator, val_generator
    
    def train(self, epochs=10, batch_size=32):
        """Train the model"""
        if self.model is None:
            self.create_model()
        
        train_gen, val_gen = self.prepare_data(batch_size)
        
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=3,
                restore_best_weights=True
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=2,
                min_lr=1e-7
            )
        ]
        
        history = self.model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=epochs,
            callbacks=callbacks
        )
        
        return history
    
    def save_model(self, path='mask_detector_model.h5'):
        """Save the trained model"""
        if self.model:
            self.model.save(path)
            print(f"Model saved to {path}")

In [ ]:
class MaskDetector:
    def __init__(self, model_path='mask_detector_model.h5'):
        """Initialize the detector"""
        self.model = keras.models.load_model(model_path)
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.classes = ['with_mask', 'without_mask']
        self.colors = {
            'with_mask': (0, 255, 0),      # Green
            'without_mask': (0, 0, 255)    # Red
        }
    
    def detect_and_predict(self, frame):
        """Detect faces and predict mask status"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.1, 
            minNeighbors=5, 
            minSize=(60, 60)
        )
        
        results = []
        
        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (224, 224))
            face_array = np.expand_dims(face_resized / 255.0, axis=0)
            
            # Predict
            predictions = self.model.predict(face_array, verbose=0)
            class_idx = np.argmax(predictions[0])
            confidence = predictions[0][class_idx]
            label = self.classes[class_idx]
            
            results.append({
                'bbox': (x, y, w, h),
                'label': label,
                'confidence': confidence
            })
        
        return results
    
    def draw_predictions(self, frame, results):
        """Draw bounding boxes and labels on frame"""
        for result in results:
            x, y, w, h = result['bbox']
            label = result['label']
            confidence = result['confidence']
            color = self.colors[label]
            
            # Draw rectangle
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
            # Draw label with confidence
            text = f"{label}: {confidence*100:.1f}%"
            cv2.putText(
                frame, text, (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2
            )
        
        return frame
    
    def run_webcam(self):
        """Run real-time detection on webcam"""
        cap = cv2.VideoCapture(0)
        
        print("Press 'q' to quit")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect and predict
            results = self.detect_and_predict(frame)
            
            # Draw results
            frame = self.draw_predictions(frame, results)
            
            # Display
            cv2.imshow('Face Mask Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
    
    def process_image(self, image_path):
        """Process a single image"""
        frame = cv2.imread(image_path)
        results = self.detect_and_predict(frame)
        frame = self.draw_predictions(frame, results)
        
        cv2.imshow('Result', frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return results


In [ ]:
if __name__ == "__main__":
    
    # TRAINING (Run this first with your dataset)

    trainer = MaskDetectionTrainer(data_dir='C:\\Users\\LENOVO\\OneDrive - King Salman International University\\Desktop\\dataset')
    trainer.create_model()
    history = trainer.train(epochs=15)
    trainer.save_model('mask_detector_model.h5')
    
    
    # REAL-TIME DETECTION (Run this after training)
    
    detector = MaskDetector(model_path='mask_detector_model.h5')
    detector.run_webcam()  # For webcam
    # OR 
    detector.process_image('path/to/image.jpg')  # For single image
    
    
    print("Face Mask Detection System Ready!")
    print("\nTo use:")
    print("1. Organize your dataset in the required structure")
    print("2. Train the model using MaskDetectionTrainer")
    print("3. Run detection using MaskDetector")

In [16]:
    detector = MaskDetector(model_path='mask_detector_model.h5')
    detector.run_webcam()  # For webcam

Press 'q' to quit
